In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import numpy as np
    
def token_lookup():
    token_dict = {".":"<period>", ",": "<comma>", ";":"<semicolon>", ":":"<colon>", "\"":"<qoutation>", "'":"qoutation", "!":"<exclamation>", "?":"<question>", 
                  "(": "<leftparan>", ")": "<rightparan>", "{":"leftbrace", "}":"<rightbrace>", "[":"<leftbracket>", "]":"<rightbracket>", "--":"<dash>", "-":"<hyphen>",
                  "\n":"<return>"}
    return token_dict

token_dict = token_lookup()

english_text = open("./plot.tok.gt9.5000").read()#open("./plot_quote.10000").read()

for key, token in token_dict.items():
    english_text = english_text.replace(key, ' {} '.format(token))
english_text = english_text.lower()
english_text = english_text.split()
english_text = ' '.join(english_text)

english_text = english_text.split("<period>")
english_text = pd.DataFrame(english_text, columns=["sentences"])

#hindi_text = pd.read_csv("./hindi_train.csv").drop(columns=['experience']).iloc[:50]
mix_text = open("./test.txt").read()#open("./codemix.txt").read()

for key, token in token_dict.items():
    mix_text = mix_text.replace(key, ' {} '.format(token))
mix_text = mix_text.lower()
mix_text = mix_text.split()
mix_text = ' '.join(mix_text)

mix_text = mix_text.split("<return>")
mix_text = pd.DataFrame(mix_text[:5000], columns=["sentences"])
mix_text.head

<bound method NDFrame.head of                                               sentences
0     <qoutation> <unk> bharat me ajaad he <comma> l...
1      indian army kashmiri logo ko nahi atankvadiyo...
2              abbe keyo fek raha hai andhe news dalal 
3      mather chooo dukan laga ke baitha rahta hai <...
4      jab modiji ne mudda utha hi diya ha to isko i...
...                                                 ...
4995   <qoutation> thappar <comma> pitai yhi <unk> k...
4996   u and u r parti candited deserve 100 slap eve...
4997   <qoutation> ek baar aap ko moka to do <comma>...
4998      in fekuo ki gand me kuchh jayada hi kida hai 
4999   <qoutation> we support aap we vote aap coming...

[5000 rows x 1 columns]>

In [ ]:
en_mix = pd.concat([english_text, mix_text], axis=0) #mix_text 
en_mix = en_mix.sample(frac = 1)
en_mix.head()

,sentences
1582,very very good movie and suspense ki to bate ...
122,aur ghuso modi k g <period> <period> <period>...
2839,<return> in their fortuitous encounter <comma...
2657,<qoutation> teri wagon r kaha hai <question> ...
5250,<return> carol miller is a psychologist with ...


In [ ]:
def create_lookup_tables(text_df):
    vocab = set()
    for line in text_df['sentences']:
        vocab.update(set(line.split()))
    #vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(en_mix)

train_int_text = []

for line in en_mix['sentences']:
    train_int_text.append([vocab_to_int[word] for word in line.split()])

train_int_text = [item for sublist in train_int_text for item in sublist]
print(train_int_text[:1000])

[14984, 14984, 1570, 17393, 13128, 9461, 4556, 1134, 7733, 3743, 16600, 17753, 4198, 16600, 16880, 11767, 6065, 22461, 4198, 4198, 4198, 4198, 18631, 4902, 12843, 10056, 4106, 15922, 8985, 6140, 16602, 10461, 12828, 6140, 10851, 14591, 22449, 6552, 22485, 16492, 13128, 12186, 3113, 23726, 2466, 21864, 18735, 19690, 3407, 21695, 456, 18426, 445, 14965, 14965, 1275, 13788, 15322, 9969, 9860, 4556, 9236, 6063, 11883, 8523, 445, 14965, 19690, 12843, 23660, 2858, 20846, 24495, 17351, 6552, 24495, 14180, 21634, 6508, 14961, 21864, 20192, 13507, 469, 6552, 16664, 4213, 2221, 2466, 427, 7997, 7557, 4198, 4198, 488, 9844, 1889, 12488, 445, 2410, 3474, 3039, 14094, 17234, 2633, 10851, 4198, 4198, 2541, 7029, 4129, 4198, 9227, 20236, 7247, 9227, 19898, 10851, 4198, 4198, 3747, 23410, 3474, 2437, 6703, 1134, 23410, 6466, 10874, 1158, 4198, 4198, 12843, 22485, 23531, 437, 24495, 640, 22363, 13392, 6205, 6140, 11003, 9885, 1134, 3195, 22485, 20434, 18976, 22780, 16783, 14358, 10851, 5975, 22485, 640

In [ ]:
print(en_mix.shape, len(vocab_to_int), max(train_int_text))

(10388, 1) 24599 24598


In [ ]:
#import helper

#data_dir = './brown.txt'

#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

#int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm)#, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 2
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_project/models/model_1_en_en/save'

batches = get_batches(train_int_text, batch_size, seq_length)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
#batches = get_batches(train_int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
#helper.save_params((seq_length, save_dir))

     |████████████████████████████████| 706kB 16.8MB/s 
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/pyth

Epoch   0 Batch    0/168   train_loss = 10.110
Epoch   0 Batch   50/168   train_loss = 7.921
Epoch   0 Batch  100/168   train_loss = 7.588
Epoch   0 Batch  150/168   train_loss = 7.479
Epoch   1 Batch   32/168   train_loss = 7.201
Epoch   1 Batch   82/168   train_loss = 7.012
Epoch   1 Batch  132/168   train_loss = 6.829
Epoch   2 Batch   14/168   train_loss = 6.593
Epoch   2 Batch   64/168   train_loss = 6.663
Epoch   2 Batch  114/168   train_loss = 6.660
Epoch   2 Batch  164/168   train_loss = 6.376
Epoch   3 Batch   46/168   train_loss = 6.433
Epoch   3 Batch   96/168   train_loss = 6.226
Epoch   3 Batch  146/168   train_loss = 6.123
Epoch   4 Batch   28/168   train_loss = 6.262
Epoch   4 Batch   78/168   train_loss = 6.147
Epoch   4 Batch  128/168   train_loss = 5.928
Epoch   5 Batch   10/168   train_loss = 6.054
Epoch   5 Batch   60/168   train_loss = 5.822
Epoch   5 Batch  110/168   train_loss = 5.784
Epoch   5 Batch  160/168   train_loss = 5.805
Epoch   6 Batch   42/168   train_

In [ ]:
import tensorflow as tf
import numpy as np
#import helper

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = np.where(probabilities == np.max(probabilities))[-1][0]

    return int_to_vocab[index]

#_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = 15, '/content/drive/MyDrive/nlp_project/models/model_1_en_en/save'
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

'''
test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")
'''
#all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    gen_length = 4000
    prime_word = 'bharat' #'यही' #'moe_szyslak'
    gen_sentences = [prime_word]
            
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
          
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        if key=='.':
            tv_script = tv_script.replace(' ' + token.lower(), '\n')
        else:
            ending = ' ' if key in ['\n', '(', '"'] else ''
            tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/nlp_project/models/model_1_en_en/save
bharat hai sir ji

me ye <unk> hai, <unk> tu tum mat hoga <unk> dub ho ki desh me jo kyu hi diya hain ki jo asliyat abhi log samjhenge ki meri apne pura kare hi nahi to jaise rules jaha log desh ko support kar rahe hin

jab mla take punjab chalane ki kaam krna haiand now, in an dogs, he was turned into the scary and childhood and run to the same time for his father' s to it outhe is the story of a young girl who have about play a luck at the world of their passion- a new war of middle class on the store of people as a group of prostitutes' s death as one of his own, but tried to live him the other plans of successlilli grew into an internet moments of a bizarre world picnic; this documentary about b small town show sir arvind kejriwal" to see with you sir <unk> will <unk>(ak) out of india <unk> v ctusip m b 1 nd plan

thank you sirab kuch nhi <unk> jaagti hai kya
" the brideshe was a 